In [6]:
import os
import urllib.request
import pandas as pd
from functools import reduce

In [44]:
years = list(range(2013, 2021))
chambers = ['house', 'senate']

metrics = ['cosponsored', 'bills-introduced', 'bills-reported',
           'committee-positions', 'cosponsors', 'ideology',
           'cosponsored-other-party', 'bills-enacted-ti', 'leadership',
           'missed-votes', 'bills-with-committee-leaders',
           'bills-with-companion', 'bills-with-cosponsor-other-party']

i_cols = [3, 5]  # columns to keep (just the metric and the bioguide)

year_dfs = []
for y in years:
    full_congress = []
    for c in chambers:
        dfs = []
        for m in metrics:
            url = f'https://www.govtrack.us/congress/members/report-cards/{y}/{c}/{m}.csv'
            try:
                dfs.append(pd.read_csv(url))
            except Exception as e:
                print(e, 'could not get csv for metric ', m, 'at url', url)

        if len(dfs) > 0:
            full_congress.append(
                reduce(
                    lambda left, right: pd.merge(left,
                                                 right,
                                                 on=['bioguide_id'],
                                                 how='inner'),
                    [x.iloc[:, i_cols] for x in dfs]
                )
            )
        else:
            print('empty dataframe list for year', y)

    if len(full_congress) > 0:
        (
            pd.concat(full_congress)
              .set_index('bioguide_id')
              .to_json(f'{y}-report-card.json', orient='index')
        )

HTTP Error 404: Not Found could not get csv for metric  bills-enacted-ti at url https://www.govtrack.us/congress/members/report-cards/2013/house/bills-enacted-ti.csv
skipping for now
HTTP Error 404: Not Found could not get csv for metric  bills-with-cosponsor-other-party at url https://www.govtrack.us/congress/members/report-cards/2013/house/bills-with-cosponsor-other-party.csv
skipping for now
HTTP Error 404: Not Found could not get csv for metric  bills-enacted-ti at url https://www.govtrack.us/congress/members/report-cards/2013/senate/bills-enacted-ti.csv
skipping for now
HTTP Error 404: Not Found could not get csv for metric  bills-with-cosponsor-other-party at url https://www.govtrack.us/congress/members/report-cards/2013/senate/bills-with-cosponsor-other-party.csv
skipping for now
HTTP Error 404: Not Found could not get csv for metric  bills-enacted-ti at url https://www.govtrack.us/congress/members/report-cards/2014/house/bills-enacted-ti.csv
skipping for now
HTTP Error 404: Not

In [52]:
import yaml, json, datetime

In [55]:
with open('misconduct.yaml', 'r') as yaml_in, open('misconduct.json', 'w') as json_out:
    yaml_object = yaml.safe_load(yaml_in)
    restructured = {x['person']: x for x in yaml_object}
    def converter(obj):
        if isinstance(obj, (datetime.datetime, datetime.date)):
            return obj.isoformat()
        raise TypeError (f"{type(obj)} not datetime")
    
    json.dump(restructured, json_out, default=converter)